<a href="https://colab.research.google.com/github/junieberry/NLP-withPyTorch/blob/main/06_SequenceModeling_forBeginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

자연어는 시계열 데이터이다!

따라서 자연어 처리를 위해 시퀀스 모델링이 필요!

시퀀스 모델링은 **은닉 상태**를 유지한다.

은닉 상태는 지금까지의 시퀀스에 대한 정보를 **시퀀스 표현**에 담는다.

시퀀스 모델로 시퀀스를 분류하거나, 생성할 수 있당

이번에는 시퀀스 분류를 배워보자~~

In [1]:
import torch
import torch.nn as nn

## 6.1 순환 신경망 소개


- t에서는 타임 스텝 t에서의 입력 벡터와 타임 스텝 t-1에서의 은닉 벡터를 사용한다.
- 각각 은닉-은닉 가중치와 입력-은닉 가중치를 사용한다.
- 이때 가중치는어느 시간에서나 같은 파라미터를 공유한다.

- 시퀀스 모델은 가변 길이 시퀀스를 다룰 수 있어야 한다.
  - 책에서는 마스킹을 사용한다.
  - 마스킹을 통해 그레디언트나 최종 출력에 포함되어서는 안된다는 신호를 보낼 수 있다.

### 6.1.1 엘만 RNN 구현하기

파이토치 RNN 클래스가 엘만 RNN의 구현이다.

하지만 여기서는 RNNCell을 사용해 RNN을 만든다!

**RNNCell**
- 입력-은닉 가중치와 은닉-은닉 가중치
- RNNCell() 호출마다 입력 벡터 행렬과 은닉 벡터 행렬을 받는다.
- 그리고 은닉 벡터를 반환한다.

In [5]:
class ElmanRNN(nn.Module):

  ## input_size (int) == 입력 벡터 크기
  ## hidden_size (int) == 은닉 벡터 크기
  ## batch_first (boolean) == 0번째 차원이 배치인가?
  def __init__(self, input_size, hidden_size, batch_first=False):
    super(ElmanRNN, self).__init__()

    ## RNNCell 사용
    self.rnn_cell = nn.RNNCell(input_size, hidden_size)

    self.batch_first = batch_first
    self.hidden_size = hidden_size
  
  def _initialize_hidden(self, batch_size):
    return torch.zeros(batch_size, self.hidden_size)
  

  # ElmanRNN의 정방향 계산
  ## x_int (tensor) == 입력 데이터 텐서
    # 만약 0번째 차원이 배치라면 (batch_size, seq_size, feat_size)
    # 아니라면 (seq_size, batch_size, feat_size)
  def forward(self, x_in, initial_hidden=None):

    if self.batch_first:
      batch_size, seq_size, feat_size = x_in.size()
      x_in = x_in.permute(1,0,2)
    else:
      seq_size, batch_size, feat_size = x_in.size()
    
    hiddens = []


    ## 초기 은닉 상태를 지정하지 않으면 모두 0
    if initial_hidden is None:
      initial_hidden = self._initialize_hidden(batch_size)
      initial_hidden = initial_hidden.to(x_in.device)
    
    hidden_t = initial_hidden

    ## 입력 벡터의 길이만큼 반복하면서 새로운 은닉 상태를 계산해서 쌓아둠
    for t in range(seq_size):
      hidden_t = self.rnn_cell(x_in[t], hidden_t)
      hiddens = torch.stack(hiddens)
    
    hiddens = torch.stack(hiddens)

    if self.batch_first:
      hiddens = hiddens.permute(1,0,2)
    
    ## 3차원 : 각 데이터 포인트/타임 스텝에 대한 은닉 상태 벡터
    # 1. 각 타임 스텝을 정해진 범주로 분류
    # 2. 최종 벡터를 사용해 전체 시퀀스를 분류
    return hiddens

In [4]:
x=torch.Tensor([[1,2,3],[1,2,3]])

a,b=x.size()
print(a,b)

2 3


https://davian-lab-junwoo.tistory.com/17

permute와 view

## 6.2 문자 RNN으로 성씨 국적 분류하기

## 6.3 요약

시퀀스 모델링의 목표는 시퀀스에 대한 표현을 학습하는 것이에용..